In [1]:
# This notebook should be ran at colab. Huggingface does not like local computing power.

# Installing required libs

In [1]:
!pip install transformers
!pip install datasets
!pip install tensorflow-addons
!pip install scikit-optimize

# Load Required libs

In [2]:
from google.colab import drive
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras import backend as K
from keras import metrics
import tensorflow_addons as tfa
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from skopt import *
import pandas as pd
from sklearn.metrics import f1_score

In [3]:
# Mount drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load data

In [ ]:
# Replace value of BASE_PATH by the directory in which the data is located.
BASE_PATH='/content/drive/MyDrive/Thesis/RobBert/'

In [4]:
with open('texts.pkl', 'rb') as f:
  text_list = pickle.load(f)
with open('labels.pkl', 'rb') as f:
  label_list = pickle.load(f)

# Model functions

In [2]:
def to_optimize(input_lst):
  '''
  This function takes a list of hyperparameters, and outputs the negative macro-averaged F1 score.
  '''
  # Params:
  # Optimizer function
  opt_function = input_lst[0]
  # Add in learning rate
  opt_function = opt_function(learning_rate=input_lst[1])
  take_weights = True
  batchsize = input_lst[2]
  num_epochs = input_lst[3]
  print('clear sessions')
  # We train several models in succession, so we need to reset the tensorflow graph
  K.clear_session()
  tf.compat.v1.reset_default_graph
  tf.compat.v1.Session()
  # Inititalize model & tokenizer
  print('begin downloaden tokenizer')
  tokenizer = RobertaTokenizer.from_pretrained("pdelobelle/robbert-v2-dutch-base")
  print('begin downloaden model')
  model = TFRobertaForSequenceClassification.from_pretrained("pdelobelle/robbert-v2-dutch-base", num_labels = 9)
  print('downloaden klaar')
  # Encode labels into integers
  le = LabelEncoder()
  labels = le.fit_transform(label_list)

  # Split into train, test, and validation set
  train_texts, test_texts, train_labels, test_labels = train_test_split(text_list, labels, test_size=.2, random_state = 420)
  train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size = 0.2, random_state = 420)

  # Tokenize text 
  train_encodings = tokenizer(train_texts, truncation=True, padding=True)
  test_encodings = tokenizer(test_texts, truncation=True, padding=True)
  val_encodings = tokenizer(val_texts, truncation=True, padding=True)

  # Transform data into tensorflow-dataset objects
  train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
  )).shuffle(len(train_texts))
  test_dataset = tf.data.Dataset.from_tensor_slices((
      dict(test_encodings),
      test_labels
  )).shuffle(len(test_texts))
  val_dataset = tf.data.Dataset.from_tensor_slices((
      dict(val_encodings),
      val_labels
  )).shuffle(len(val_texts))
  # Define the model
  model.compile(optimizer=opt_function, 
                loss=model.compute_loss, 
                metrics=['accuracy'] 
                ) 
  # Start Training
  print('--------------------STARTING TRAINING--------------------')
  print(f'Optimization function: {opt_function}')
  print(f'Learning rate: {input_lst[1]}')
  print(f'Batch size: {batchsize}')
  if take_weights == False:
    history = model.fit(train_dataset.batch(batchsize), 
                        epochs=num_epochs, 
                        batch_size=batchsize, 
                        validation_data=val_dataset.batch(batchsize)
                        )
  else:
    class_weights = dict(enumerate(compute_class_weight('balanced', np.unique(labels), labels)))
    history = model.fit(train_dataset.batch(batchsize), 
                        epochs=num_epochs, batch_size=batchsize, 
                        validation_data=val_dataset.batch(batchsize), 
                        class_weight=class_weights
                        )
  
  model.save_pretrained(BASE_PATH+f"bayesianopt_epochs{num_epochs}_BS{batchsize}_OPT{opt_function}_LR{input_lst[1]}")
  # Save history as well for plotting later on
  with open(BASE_PATH+f"HISTORY_bayesianopt_epochs{num_epochs}_BS{batchsize}_OPT{opt_function}_LR{input_lst[1]}", 'wb') as f:
    pickle.dump(history.history, f)
  # model.predict outputs logits, so we need to pass them through a softmax layer
  predictions = tf.nn.softmax(model.predict(test_dataset.batch(batchsize)).logits)
  # Take labels out of the tensorflow dataset object
  test_labels = np.concatenate([y for x, y in test_dataset.batch(batchsize)], axis=0)
  # Calculate the macro-averaged f1 score.
  f1_macro = f1_score(test_labels, np.argmax(predictions, axis=1), average='macro')
  print(f"macro f1_score: {f1_macro}")
  # Delete model from memory
  del model
  return -f1_macro

In [6]:
# This cell defines the search space for the bayesian optimization
d_optimizer = space.Categorical(categories=[tf.keras.optimizers.Adam, 
                                            tf.keras.optimizers.SGD,
                                            tf.keras.optimizers.Adamax,
                                            ], name='optimizer function')
d_learning_rate = space.Real(low=1e-5, high=1e-1, name='learing rate')
d_batchsize = space.Integer(low=4, high=6, name='batch size')
d_epochs = space.Integer(low=1, high=10, name='number of epochs')

In [7]:
# Define the dimensions, these will be put in the model function (to_optimize)
dimensions = [d_optimizer, d_learning_rate, d_batchsize, d_epochs]
# Define the starting set of HP's
default = [tf.keras.optimizers.SGD, 1e-5, 4, 10]

In [8]:
# Run Bayesian optimization.
gp_results = gp_minimize(func=to_optimize, dimensions=dimensions, acq_func='EI', random_state=1, x0=default, n_calls=25)

clear sessions
begin downloaden tokenizer
begin downloaden model


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at pdelobelle/robbert-v2-dutch-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


downloaden klaar
--------------------STARTING TRAINING--------------------
Optimization function: <tensorflow.python.keras.optimizer_v2.gradient_descent.SGD object at 0x7f61c298d790>
Learning rate: 1e-05
Batch size: 4
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_func

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at pdelobelle/robbert-v2-dutch-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


downloaden klaar
--------------------STARTING TRAINING--------------------
Optimization function: <tensorflow.python.keras.optimizer_v2.adamax.Adamax object at 0x7f61c28e4a10>
Learning rate: 0.09325641036027252
Batch size: 4
Epoch 1/10
191/191 [==============================] - 68s 268ms/step - loss: 19.5061 - accuracy: 0.0995 - val_loss: 8.3593 - val_accuracy: 0.1466
Epoch 2/10
191/191 [==============================] - 48s 252ms/step - loss: 16.0879 - accuracy: 0.1139 - val_loss: 7.1816 - val_accuracy: 0.2513
Epoch 3/10
191/191 [==============================] - 48s 253ms/step - loss: 17.9845 - accuracy: 0.0995 - val_loss: 32.9524 - val_accuracy: 0.0262
Epoch 4/10
191/191 [==============================] - 48s 253ms/step - loss: 17.8145 - accuracy: 0.1086 - val_loss: 12.1198 - val_accuracy: 0.1466
Epoch 5/10
191/191 [==============================] - 48s 253ms/step - loss: 17.0090 - accuracy: 0.1165 - val_loss: 6.0914 - val_accuracy: 0.1518
Epoch 6/10
191/191 [=======================

...

downloaden klaar
--------------------STARTING TRAINING--------------------
Optimization function: <tensorflow.python.keras.optimizer_v2.adam.Adam object at 0x7f610890a0d0>
Learning rate: 0.09839421140639927
Batch size: 4
Epoch 1/10
191/191 [==============================] - 68s 270ms/step - loss: 83.7861 - accuracy: 0.1021 - val_loss: 83.7431 - val_accuracy: 0.1990
Epoch 2/10
191/191 [==============================] - 49s 255ms/step - loss: 100.0473 - accuracy: 0.1414 - val_loss: 216.8045 - val_accuracy: 0.0105
Epoch 3/10
191/191 [==============================] - 49s 255ms/step - loss: 89.5484 - accuracy: 0.0720 - val_loss: 90.1326 - val_accuracy: 0.1518
Epoch 4/10
191/191 [==============================] - 49s 255ms/step - loss: 103.6669 - accuracy: 0.1335 - val_loss: 131.6423 - val_accuracy: 0.1990
Epoch 5/10
191/191 [==============================] - 49s 255ms/step - loss: 86.4270 - accuracy: 0.1139 - val_loss: 25.7243 - val_accuracy: 0.1466
Epoch 6/10
191/191 [====================

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at pdelobelle/robbert-v2-dutch-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


downloaden klaar
--------------------STARTING TRAINING--------------------
Optimization function: <tensorflow.python.keras.optimizer_v2.adamax.Adamax object at 0x7f610710f190>
Learning rate: 0.09999607299165171
Batch size: 5
Epoch 1/10
153/153 [==============================] - 66s 322ms/step - loss: 23.7695 - accuracy: 0.1126 - val_loss: 18.1088 - val_accuracy: 0.0785
Epoch 2/10
153/153 [==============================] - 46s 301ms/step - loss: 15.7515 - accuracy: 0.1099 - val_loss: 6.9642 - val_accuracy: 0.1518
Epoch 3/10
153/153 [==============================] - 46s 302ms/step - loss: 15.0045 - accuracy: 0.0955 - val_loss: 9.7979 - val_accuracy: 0.1990
Epoch 4/10
153/153 [==============================] - 46s 301ms/step - loss: 18.6392 - accuracy: 0.1387 - val_loss: 7.5946 - val_accuracy: 0.2513
Epoch 5/10
153/153 [==============================] - 46s 301ms/step - loss: 11.1085 - accuracy: 0.1191 - val_loss: 5.3586 - val_accuracy: 0.0576
Epoch 6/10
153/153 [========================

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at pdelobelle/robbert-v2-dutch-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


downloaden klaar
--------------------STARTING TRAINING--------------------
Optimization function: <tensorflow.python.keras.optimizer_v2.adam.Adam object at 0x7f5f44ab0190>
Learning rate: 0.00010125993823601541
Batch size: 4
Epoch 1/10
191/191 [==============================] - 69s 270ms/step - loss: 2.1352 - accuracy: 0.2448 - val_loss: 2.0088 - val_accuracy: 0.2461
Epoch 2/10
191/191 [==============================] - 49s 254ms/step - loss: 2.3122 - accuracy: 0.0537 - val_loss: 2.1572 - val_accuracy: 0.1518
Epoch 3/10
191/191 [==============================] - 49s 254ms/step - loss: 2.2498 - accuracy: 0.0589 - val_loss: 2.2434 - val_accuracy: 0.0105
Epoch 4/10
191/191 [==============================] - 49s 255ms/step - loss: 2.2164 - accuracy: 0.0903 - val_loss: 2.0572 - val_accuracy: 0.1518
Epoch 5/10
191/191 [==============================] - 49s 255ms/step - loss: 2.2284 - accuracy: 0.1387 - val_loss: 2.2018 - val_accuracy: 0.0785
Epoch 6/10
191/191 [==============================]

In [9]:
with open(BASE_PATH+'gp_results_final.pkl', 'wb') as f:
  pickle.dump(gp_results, f)

In [10]:
results_df = pd.DataFrame(columns = ['optimizer', 'learning_rate', 'batch_size', 'num_epochs'])
for i in gp_results['x_iters']:
  series = pd.Series(i, index = results_df.columns)
  results_df = results_df.append(series, ignore_index=True)

In [11]:
results_df['-f1'] = pd.Series(gp_results['func_vals'])

In [12]:
results_df['f1'] = results_df['-f1'].apply(lambda x: -x)

In [13]:
results_df.sort_values('f1', ascending=False).to_excel(BASE_PATH+'bayesian_results.xlsx')

In [14]:
with open(BASE_PATH+'gp_results_final.pkl', 'rb') as f:
  gp_results = pickle.load(f)

In [16]:
gp_results['x']

[tensorflow.python.keras.optimizer_v2.adamax.Adamax,
 0.09626070959915467,
 4,
 10]